# Week 10 — Functional Pipelines + Testing Basics

**Time budget:** ~2 hours  
**Goal:** Build transformation pipelines (map/filter) and write a few unit tests for parser functions.

**Theme (PhD focus):** Human factors of privacy & security — scraping public pages (privacy policies, cookie notices, security help pages, standards/regulator guidance) and extracting *UX-relevant* signals.

---


## Deliverables
- A completed notebook with working code
- A dataset variable (`rows` or `df`) saved to disk (CSV/JSON depending on week)
- 3–5 bullet reflection grounded in human factors/privacy-security research


## Responsible scraping note (important)
We will only scrape **public pages** and keep the volume small.
- Prefer a few pages, not thousands
- Respect robots.txt/Terms of Service when you scale later
- Avoid collecting personal data
- Add delays for politeness when doing multi-page work


## Step 0 — Imports

In [ ]:
import re
import time
import json
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup

import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass

## Step 1 — Write a pure function and test it

In [ ]:
def parse_cues(text: str) -> dict:
    return {
        "choices_controls": bool(re.search(r"\b(opt\s?-?out|preferences|your choices|controls?)\b", text, re.I)),
        "retention": bool(re.search(r"\b(retention|retain)\b", text, re.I)),
    }

def test_parse_cues():
    t = "You can opt out in settings. We retain data for 30 days."
    cues = parse_cues(t)
    assert cues["choices_controls"] is True
    assert cues["retention"] is True

test_parse_cues()
print("OK")

## Step 2 — Functional pipeline over URLs

In [ ]:
def fetch_text(url: str) -> str:
    r = requests.get(url, timeout=20)
    soup = BeautifulSoup(r.text, "html.parser")
    return soup.get_text(" ", strip=True)

urls = [
    "https://www.mozilla.org/en-US/privacy/",
    "https://www.nist.gov/privacy-framework",
]
rows = [{"url": u, **parse_cues(fetch_text(u))} for u in urls]
pd.DataFrame(rows)

## Reflection: testability

- Why are pure functions easier to test?
